In [1]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars
!pip install keras-rl2
!pip install gym
!pip install gym[atari]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
copying adventure.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Adventure (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Air Raid (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Alien.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying crazy_climber.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Crazy Climber.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/crazy_climber.bin
copying elevator_action.bin from HC ROMS/BY ALPHABET (PAL)/

In [8]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
import gym
import numpy as np
# import tensorflow as tf
import tensorflow.keras as k
# import keras as k
# import keras.layers as layers
import tensorflow.keras.layers as layers
# these only necessary in colab for visualization
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

In [ ]:
# example for displaying game (slow)
# import gym
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()
# Create a breakout environment
# env = gym.make('BreakoutDeterministic-v4')
env = gym.make("Breakout-v0")
# Reset it, returns the starting frame
frame = env.reset()
input_shape = env.observation_space.shape
prev_screen = env.render(mode='rgb_array')
#env.render()
plt.imshow(prev_screen)

is_done = False
while not is_done:
  # Perform a random action, returns the new frame, reward and whether the game is over
  frame, reward, is_done, _ = env.step(env.action_space.sample())
  # Render
  screen = env.render(mode='rgb_array')
  # uncomment next line to view
  # plt.imshow(screen)
  ipythondisplay.clear_output(wait=True)
  ipythondisplay.display(plt.gcf())

In [14]:
env = gym.make("Breakout-v0")
num_actions = env.action_space.n
state_size = env.observation_space.shape

def build_model(state_size, num_actions):
    """ build the keras model for deep learning """
    # inputs = layers.Input(shape=(84, 84, 4,))
    inputs = layers.Input(shape=(4,) + state_size)
    layer1 = layers.Conv2D(32, 8, strides=4, activation="relu")(inputs)
    layer2 = layers.Conv2D(64, 4, strides=2, activation="relu")(layer1)
    layer3 = layers.Conv2D(64, 3, strides=1, activation="relu")(layer2)
    layer4 = layers.Flatten()(layer3)
    layer5 = layers.Dense(512, activation="relu")(layer4)
    action = layers.Dense(num_actions, activation="linear")(layer5)
    return k.Model(inputs=inputs, outputs=action)

model = build_model(state_size, num_actions)
model.summary()



Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4, 210, 160, 3)]  0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 4, 22, 16, 64)     36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 90112)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               46137856  
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 2052

In [15]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.callbacks import FileLogger

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1.,
                              value_min=.1, value_test=0.1, nb_steps=1000000)
memory = SequentialMemory(limit=1000000, window_length=4)
agent = DQNAgent(model=model, policy=policy, nb_actions=num_actions,
                 memory=memory, nb_steps_warmup=50000)
agent.compile(k.optimizers.Adam(learning_rate=.00025), metrics=['mae'])
agent.fit(env, nb_steps=10000, log_interval=1000, visualize=False, verbose=2)

Training for 10000 steps ...


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  170/10000: episode: 1, duration: 8.321s, episode steps: 170, steps per second:  20, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.300 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  367/10000: episode: 2, duration: 8.885s, episode steps: 197, steps per second:  22, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.604 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  760/10000: episode: 3, duration: 16.829s, episode steps: 393, steps per second:  23, episode reward:  4.000, mean reward:  0.010 [ 0.000,  1.000], mean action: 1.466 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  937/10000: episode: 4, duration: 7.434s, episode steps: 177, steps per second:  24, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.525 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
 1109/10000: episode: 5, duration: 7.088s, episode steps: 172, steps per second:  24, e

In [17]:
agent.save_weights('policy.h5', overwrite=True)
agent.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 0.000, steps: 10000
Episode 2: reward: 0.000, steps: 10000
Episode 3: reward: 0.000, steps: 10000
Episode 4: reward: 0.000, steps: 10000
Episode 5: reward: 0.000, steps: 10000
Episode 6: reward: 0.000, steps: 10000
Episode 7: reward: 0.000, steps: 10000
Episode 8: reward: 0.000, steps: 10000
Episode 9: reward: 0.000, steps: 10000
Episode 10: reward: 0.000, steps: 10000
